In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime, timezone
import time

from typing import Dict, List, Optional, Union

from osrs_filter import OSRSItemFilter

from llm.client import ModelConfig, get_chat_response, init_client, remove_thinking_tags
from utils.config import settings
# Configuration
BASE_URL = "https://prices.runescape.wiki/api/v1/osrs"

# IMPORTANT: Set a descriptive User-Agent as required by the API
# Replace with your own descriptive user agent
USER_AGENT = "learning pandas with osrs - @sf1tzp"

# Default headers with proper User-Agent
HEADERS = {
    'User-Agent': USER_AGENT,
    'Accept': 'application/json'
}

print(f"Using User-Agent: {USER_AGENT}")
print(f"Base URL: {BASE_URL}")

In [ ]:
# Reinitialize with forced reload to clear cached timezone-naive data
print("Reinitializing OSRSItemFilter with fresh data...")
item_filter = OSRSItemFilter(user_agent=USER_AGENT)
item_filter.load_data(force_reload=True)
item_filter.apply_filter(
    margin_pct_min=3,
    sold_price_max=3200000,
    sold_price_min=80000,
    max_hours_since_update=0.25,
    sort_by=("margin_gp", "desc"),
    limit=300

)
print(f"Requsting volume data at: {datetime.now()}")
item_filter.load_volume_data(max_items=10000)
print(f"Volume data fetching complete at: {datetime.now()}")

# Example of saving filtered data to files in different formats
print("Saving filtered data to files...")

# Save to Pickle (best for preserving all data types)
item_filter.save("filtered_items.pkl", format="pickle")

In [ ]:
# item_filter.save("filtered_items.pkl", format="pickle")

In [ ]:
loaded_filter = OSRSItemFilter(user_agent=USER_AGENT)
loaded_filter.load_from_file("filtered_items.pkl", format="pickle")
loaded_filter.apply_filter(
    volume_24h_min=500,
    volume_1h_min=100
)

with open("loaded_filter.txt", "w") as file:
    file.write(str(loaded_filter))

# Save with improved formatting for LLM
loaded_filter.save("loaded_filter.json", format="json")


loaded_filter

In [ ]:


# note: our bot will not save & load intermediate data to disk
with open("loaded_filter.json", "r") as f:
    data_json = f.read()


with open("over-here!.txt", "r") as f:
    data_text = f.read()



In [ ]:


# Prototype LLM interaction using get_generate_response
from llm.client import ModelConfig, get_generate_response
# Configure qwen3:14b with 10k token context
model_config = ModelConfig.create_default("qwen3:14b")
model_config.options.num_ctx = 20000  # 10k token context

# Read the prompt.txt file as system prompt
with open("../prompt.md", "r") as f:
    prompt = f.read()

with open("../signals.md", "r") as f:
    signals = f.read()

with open("../example-output.md", "r") as f:
    example = f.read()

full_prompt=f"""
{prompt}

<signals>
{signals}
</signals>

<example grade="a great example">
{example}
</example>
"""


In [ ]:


user_prompt = f"{data_json}"

# Get response from LLM
response = get_generate_response(
    model_config=model_config,
    system_prompt=full_prompt,
    user_prompt=user_prompt
)

# Clean the response by removing thinking tags
cleaned_response = remove_thinking_tags(response.response)

print("Cleaned response (without thinking tags):")
print(f"Generated at: {datetime.now()}\n")
print("=" * 50)
print(cleaned_response)

# Save the cleaned response to a separate file
with open("llm_trading_analysis_json.md", "w") as f:
    f.write(cleaned_response)


In [ ]:


user_prompt = f"{data_text}"

# Get response from LLM
response = get_generate_response(
    model_config=model_config,
    system_prompt=full_prompt,
    user_prompt=user_prompt
)

# Clean the response by removing thinking tags
cleaned_response = remove_thinking_tags(response.response)

print("Cleaned response (without thinking tags):")
print(f"Generated at: {datetime.now()}\n")
print("=" * 50)
print(cleaned_response)

# Save the cleaned response to a separate file
with open("llm_trading_analysis_text.md", "w") as f:
    f.write(cleaned_response)